In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pykrx import stock
import io
import boto3

In [2]:
korea = pd.read_csv('국내_산업분류.csv', encoding = 'cp949')
korea['종목코드'] = korea['종목코드'].str.replace("'", '')
korea

,소분류,기업,종목코드,시장
0,Specialty Chemicals,탑머티리얼,360070,KOSDAQ
1,Semiconductor Equipment & Materials,삼성전자,005930,KOSPI
2,Semiconductor Equipment & Materials,SK하이닉스,000660,KOSPI
3,Semiconductor Equipment & Materials,원익IPS,240810,KOSDAQ GLOBAL
4,Semiconductor Equipment & Materials,피에스케이홀딩스,031980,KOSDAQ
...,...,...,...,...
4280,Confectioners,빙그레,005180,KOSPI
4281,Confectioners,오뚜기,007310,KOSPI
4282,Confectioners,대한제분,001130,KOSPI
4283,Confectioners,대상,001680,KOSPI


In [3]:
# 종목코드를 주가 데이터 크롤링용으로 변환
new_korea = korea.copy()
def change_ticker(row):
    if 'KOSDAQ' in row['시장']:
        return row['종목코드'].replace("'", '') + '.KQ'
    elif 'KOSPI' in row['시장']:
        return row['종목코드'].replace("'", '') + '.KS'
    else:
        return row['종목코드'].replace("'", '')

new_korea['종목코드'] = new_korea.apply(change_ticker, axis = 1)
new_korea

,소분류,기업,종목코드,시장
0,Specialty Chemicals,탑머티리얼,360070.KQ,KOSDAQ
1,Semiconductor Equipment & Materials,삼성전자,005930.KS,KOSPI
2,Semiconductor Equipment & Materials,SK하이닉스,000660.KS,KOSPI
3,Semiconductor Equipment & Materials,원익IPS,240810.KQ,KOSDAQ GLOBAL
4,Semiconductor Equipment & Materials,피에스케이홀딩스,031980.KQ,KOSDAQ
...,...,...,...,...
4280,Confectioners,빙그레,005180.KS,KOSPI
4281,Confectioners,오뚜기,007310.KS,KOSPI
4282,Confectioners,대한제분,001130.KS,KOSPI
4283,Confectioners,대상,001680.KS,KOSPI


In [6]:
korea

,소분류,기업,종목코드,시장
0,Specialty Chemicals,탑머티리얼,360070,KOSDAQ
1,Semiconductor Equipment & Materials,삼성전자,005930,KOSPI
2,Semiconductor Equipment & Materials,SK하이닉스,000660,KOSPI
3,Semiconductor Equipment & Materials,원익IPS,240810,KOSDAQ GLOBAL
4,Semiconductor Equipment & Materials,피에스케이홀딩스,031980,KOSDAQ
...,...,...,...,...
4280,Confectioners,빙그레,005180,KOSPI
4281,Confectioners,오뚜기,007310,KOSPI
4282,Confectioners,대한제분,001130,KOSPI
4283,Confectioners,대상,001680,KOSPI


In [7]:
def collect_stock_data(ticker, start_date, end_date):
    # KONEX의 경우 pykrx에 데이터가 있음
    if new_korea.loc[new_korea['종목코드'] == ticker, '시장'].values[0] == 'KONEX':
        data = stock.get_market_ohlcv_by_date(fromdate=start_date.strftime('%Y%m%d'), todate=end_date.strftime('%Y%m%d'), ticker=ticker)
        data.reset_index(inplace = True)
        data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change']
        data['Ticker'] = ticker
        result = data[1:]  # 첫 번째 행 제거 - 첫 번째 행은 2년 전 해당일의 등락률 계산을 위해서만 사용
    else:
        try:
            data = yf.download(ticker, start=start_date, end=end_date)
            if not data.empty and 'Open' in data.columns:
                data['Change'] = data['Adj Close'].pct_change() * 100  # 등락률 계산
                data = data.drop('Adj Close', axis = 1)
                data['Ticker'] = ticker.split('.')[0]  # ticker 값에 '.KS' 부분 제거
                result = data[1:]  # 첫 번째 행 제거
            else:
                raise Exception("Data from yf.download is empty or missing 'Adj Close' column.")
        except Exception as e:
            ticker_new = ticker.split('.')[0]
            print(f"Error for {ticker}: {e}")
            data = stock.get_market_ohlcv_by_date(fromdate=start_date.strftime('%Y%m%d'), todate=end_date.strftime('%Y%m%d'), ticker=ticker_new)
            data.reset_index(inplace = True)
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change']
            data['Ticker'] = ticker_new
            result = data[1:]  # 첫 번째 행 제거 - 첫 번째 행은 2년 전 해당일의 등락률 계산을 위해서만 사용

    return result

def upload(tickers, start_date, end_date, s3_client, bucket_name, endpoint_url,
                                  access_key, secret_key):
    failed_ls = []
    
    for ticker in tickers:
        data = collect_stock_data(ticker, start_date, end_date)
        
        # '시가' 열의 값이 모두 0인지 확인
        if data['Open'].eq(0).all():
            print(f"Data for {ticker} has all 'Open' values as 0. Skipping upload.")
            failed_ls.append(ticker)
            continue
        
        # 기업명 가져오기
        index = new_korea[new_korea['종목코드'] == ticker].index.values[0]
        company_name = korea.at[index, '기업']
        
        # 데이터프레임을 파일로 저장
        xlsx_filename = f'{company_name}_주가데이터.xlsx'
        xlsx_buffer = io.BytesIO()
        data.to_excel(xlsx_buffer)
        xlsx_buffer.seek(0)
        
        # xlsx 파일 클라우드 업로드
        object_name = f'kr_stock_crawling/{xlsx_filename}'
        s3_client.upload_fileobj(xlsx_buffer, bucket_name, object_name)
        print(f"Uploaded {xlsx_filename} to S3")
        
    print('Failed tickers: ', failed_ls)

def main():
    # AWS S3 설정
    service_name = 's3'
    endpoint_url = 'https://kr.object.ncloudstorage.com'
    access_key = 'cPKhn3d9z9ZWYMAPPwyq'
    secret_key = 'VFjEkpateO6VFGY0U9op073I4NXTlejMBMIq6FvA'
    bucket_name = 'bucket1chaegpt'
    
    # 티커 리스트 생성
    korea_tickers = new_korea.iloc[:, 2].unique()
    
    # 오늘 날짜로부터 2년 전의 날짜 계산
    end_date = datetime.today()
    start_date = datetime.today() - timedelta(days=365*2 + 1)
    
    # AWS S3 클라이언트 생성
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)

    # 클라우드 폴더 생성
    object_name = 'kr_stock_crawling/'
    s3.put_object(Bucket=bucket_name, Key=object_name)

    # 주가 데이터 수집 및 파일로 저장
    upload(korea_tickers, start_date, end_date, s3, bucket_name, endpoint_url, access_key, secret_key)

if __name__ == '__main__':
    main()

[*********************100%%**********************]  1 of 1 completed
Uploaded 탑머티리얼_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 삼성전자_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded SK하이닉스_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 원익IPS_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 피에스케이홀딩스_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 유진테크_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 테크윙_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 테스_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 원익머트리얼즈_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 원익QnC_주가데이터.xl


1 Failed download:
['033340.KQ']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-08-31 00:29:52.480365 -> 2023-09-01 00:29:52.480365)')



Error for 033340.KQ: Data from yf.download is empty or missing 'Adj Close' column.
Data for 033340.KQ has all 'Open' values as 0. Skipping upload.
[*********************100%%**********************]  1 of 1 completed
Uploaded 삼기이브이_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 중앙디앤엠_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 엠아이큐브솔루션_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 동국홀딩스_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 에이엘티_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 파로스아이바이오_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 인벤티지랩_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 큐라클_주가데이터.xlsx to S3
[*********************100%%**************


1 Failed download:
['065560.KQ']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-08-31 00:29:52.480365 -> 2023-09-01 00:29:52.480365)')



Error for 065560.KQ: Data from yf.download is empty or missing 'Adj Close' column.
Data for 065560.KQ has all 'Open' values as 0. Skipping upload.
[*********************100%%**********************]  1 of 1 completed
Uploaded 삼양통상_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 신스틸_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 브릿지바이오테라퓨틱스_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 버넥트_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 샌즈랩_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 조선선재_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 마음AI_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 삼영_주가데이터.xlsx to S3
[*********************100%%**********************]

...


Error for 058220.KQ: Data from yf.download is empty or missing 'Adj Close' column.
Data for 058220.KQ has all 'Open' values as 0. Skipping upload.
Uploaded 진코스텍_주가데이터.xlsx to S3
Uploaded 판도라티비_주가데이터.xlsx to S3
Uploaded 에스엠비나_주가데이터.xlsx to S3
Uploaded 피노텍_주가데이터.xlsx to S3
Uploaded 큐엠씨_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['121890.KQ']: Exception('%ticker%: No timezone found, symbol may be delisted')



Error for 121890.KQ: Data from yf.download is empty or missing 'Adj Close' column.
Data for 121890.KQ has all 'Open' values as 0. Skipping upload.
Uploaded 수프로_주가데이터.xlsx to S3
Uploaded 태양기계_주가데이터.xlsx to S3
Uploaded 엔에스엠_주가데이터.xlsx to S3
Uploaded 지앤이헬스케어_주가데이터.xlsx to S3
Uploaded 테크트랜스_주가데이터.xlsx to S3
Uploaded 코셋_주가데이터.xlsx to S3
Uploaded 에스엘테라퓨틱스_주가데이터.xlsx to S3
Uploaded 크로넥스_주가데이터.xlsx to S3
Uploaded 렌딩머신_주가데이터.xlsx to S3
Uploaded 질경이_주가데이터.xlsx to S3
Uploaded 마이크로엔엑스_주가데이터.xlsx to S3
Uploaded 썬테크_주가데이터.xlsx to S3
Uploaded 골프존데카_주가데이터.xlsx to S3
Uploaded 에스제이켐_주가데이터.xlsx to S3
Uploaded 에스알바이오텍_주가데이터.xlsx to S3
Uploaded 에스케이씨에스_주가데이터.xlsx to S3
Uploaded 유니포인트_주가데이터.xlsx to S3
Uploaded 파워풀엑스_주가데이터.xlsx to S3
Uploaded 대주이엔티_주가데이터.xlsx to S3
Uploaded 피엔아이컴퍼니_주가데이터.xlsx to S3
Uploaded 미래엔에듀파트너_주가데이터.xlsx to S3
Uploaded 엘리비젼_주가데이터.xlsx to S3
Uploaded 바이오인프라생명과학_주가데이터.xlsx to S3
Uploaded 지에프씨생명과학_주가데이터.xlsx to S3
Uploaded 티엘엔지니어링_주가데이터.xlsx to S3
Uploaded 뿌리깊은나무들_주가데이터.xlsx to S3
Upload


1 Failed download:
['078590.KQ']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-08-31 00:29:52.480365 -> 2023-09-01 00:29:52.480365)')



Error for 078590.KQ: Data from yf.download is empty or missing 'Adj Close' column.
Data for 078590.KQ has all 'Open' values as 0. Skipping upload.
[*********************100%%**********************]  1 of 1 completed
Uploaded 한국제지_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded HLB파나진_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded CJ씨푸드_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 넥스틸_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 자이글_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 푸른저축은행_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 덕성_주가데이터.xlsx to S3
[*********************100%%**********************]  1 of 1 completed
Uploaded 써니전자_주가데이터.xlsx to S3
[*********************100%%**********************] 

In [18]:
Failed_ls = ['053590.KQ', '078130.KQ', '062860.KQ', '014200.KQ', '214870.KQ', '066410.KQ', '056090.KQ', '136510.KQ', '318020.KQ', '150840.KQ', '057880.KQ', '033340.KQ', '138690.KQ', '065560.KQ', '215090.KQ', '178780.KQ', '160600.KQ', '269620.KQ', '257370.KQ', '226440.KQ', '006580.KQ', '032860.KQ', '069110.KQ', '263540.KQ', '181340.KQ', '099520.KQ', '141020.KQ', '038340.KQ', '058530.KQ', '148140.KQ', '219750.KQ', '299910.KQ', '208860.KQ', '114120.KQ', '214310.KQ', '058220.KQ', '121890.KQ', '078590.KQ']

In [57]:
# Failed_ls에 해당되는 종목들은 거래 정지 종목으로 제거해줌.
for ticker in Failed_ls:
    clean_ticker = ticker.split('.')[0]
    korea = korea[~(korea['종목코드'] == clean_ticker)].reset_index(drop = True)

In [58]:
korea

,소분류,기업,종목코드,시장
0,Specialty Chemicals,탑머티리얼,360070,KOSDAQ
1,Semiconductor Equipment & Materials,삼성전자,005930,KOSPI
2,Semiconductor Equipment & Materials,SK하이닉스,000660,KOSPI
3,Semiconductor Equipment & Materials,원익IPS,240810,KOSDAQ GLOBAL
4,Semiconductor Equipment & Materials,피에스케이홀딩스,031980,KOSDAQ
...,...,...,...,...
4221,Confectioners,빙그레,005180,KOSPI
4222,Confectioners,오뚜기,007310,KOSPI
4223,Confectioners,대한제분,001130,KOSPI
4224,Confectioners,대상,001680,KOSPI


In [59]:
new_korea.to_csv('국내_산업분류(추가제거).csv', index = False, encoding = 'cp949')